# Imports

In [18]:
import numpy as np, pandas as pd
import os, shutil
from distutils.dir_util import copy_tree

## Define source path for train and test data

In [33]:
'''
Define the path to the train and test data files on your local computer. 

src_path should be a directory which has two folders: 1) train, and 2) test
'train' folder must have a file called: ratings_train.csv
'test' folder must have a file called : ratings_test.csv 

Both 'ratings_train.csv' and 'ratings_test.csv' should have the following four fields (comma-separated): 
rating_id (string type)
user_id (string type)
item_id (string type)
rating (float)

You may use the test file purely for predictions, i.e. without doing scoring. In that case, the file may exclude the 'rating'
column. 
'''


dataset = 'jester'      # jester, movielens-10m, movielens-20m, book-crossing

src_path = f'./data/{dataset}/processed/'

## Make Volume to Mount with appropriate folders
This is a folder created in this present directory. <br>
We will copy the data files into this folder and then bind mount this folder into the container. <br>
When the container is run, model artifacts and other outputs will also be saved to this same folder. <br>
These files on the shared volume will persist when the container is shut down. 

In [20]:
# Run this as-is. Do not change any folder names!!! 

mounted_volume = 'ml_vol'
if os.path.exists(mounted_volume): shutil.rmtree(mounted_volume)

os.mkdir(mounted_volume)
subdirs = ['data', 'logs', 'model', 'output']
for subdir in subdirs: 
    sub_dir_path = os.path.join(mounted_volume, subdir)
    os.mkdir(sub_dir_path)

subdirs = ['train', 'test']
for subdir in subdirs: 
    sub_dir_path = os.path.join(mounted_volume, 'data', subdir)
    os.mkdir(sub_dir_path)

## Copy Data from Source Path Into Mounted Volume

In [21]:
dest_path = os.path.join(mounted_volume, 'data')
for input_type in ['train', 'test']:
    full_src = os.path.join(src_path, input_type)
    full_dest = os.path.join(dest_path, input_type)
    
    if os.path.exists(full_src): copy_tree(full_src, full_dest)

# Build Image

In [34]:
%%bash 
docker build -t abudesai/rec_base_autorec:v1.0.0 ./autorec

#1 [internal] load build definition from Dockerfile
#1 sha256:f211a46fbd51ae44331f6c2ae33a1740bb4375ebfbbe1cc754ec02a113e32fb9
#1 transferring dockerfile: 265B 0.0s done
#1 DONE 0.0s

#2 [internal] load .dockerignore
#2 sha256:dc0566532ab6161c5d5c29f1c4d80d68821a1ea4b0530eca87f8a504f5b5302e
#2 transferring context: 62B 0.0s done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/python:3.8.0-slim
#3 sha256:4f0c597550e30aa54f707f0017cf64d137017976c13b147baa6fd4ad0c55c91e
#3 ...

#4 [auth] library/python:pull token for registry-1.docker.io
#4 sha256:5ddbd5df9a439d5a8a99ee7d22480bb467e5c9b8822254a977c76ae2494f2c68
#4 DONE 0.0s

#3 [internal] load metadata for docker.io/library/python:3.8.0-slim
#3 sha256:4f0c597550e30aa54f707f0017cf64d137017976c13b147baa6fd4ad0c55c91e
#3 DONE 7.0s

#5 [1/5] FROM docker.io/library/python:3.8.0-slim@sha256:8e243f41e500238f78f7a29a81656114d3fe603d5c34079a462d090f71c4b225
#5 sha256:f2202870b184ece5b9a09c9b777f938cf0be25287ffe019e2c50e60191382ede


# Create Container From Image

In [35]:
%%bash 
declare vol_path="$(pwd)/ml_vol"
docker run -d -p 3000:3000 -v $vol_path:/app/ml_vol --name autorec abudesai/rec_base_autorec:v1.0.0

ebf4e1cc3c64a7a484290926c360bc80046a12211d75626bc27a19caf675ed33


# Check Container and Image

In [36]:
%%bash 
docker ps

CONTAINER ID   IMAGE                              COMMAND                  CREATED         STATUS         PORTS                    NAMES
ebf4e1cc3c64   abudesai/rec_base_autorec:v1.0.0   "/bin/sh -c 'python â€¦"   7 seconds ago   Up 6 seconds   0.0.0.0:3000->3000/tcp   autorec


In [37]:
#%%bash 
#docker inspect mfc

# Run Training

In [38]:
%%bash 
docker exec autorec python train.py

Starting the training process...
orig_train_data shape: (3929542, 4)
After train/valid split, train_data shape: (3536374, 4) valid_data shape: (393168, 4)
Pre-processing data...
data based params: {'M': 100}
model_params: {'M': 100, 'drop_out': 0.7, 'lr': 0.08}
Training matrix factorizer ...
Epoch 1/15
574/574 [==============================] - 7s 12ms/step - loss: 0.0587 - custom_loss: 0.0587 - val_loss: 0.0493 - val_custom_loss: 0.0493
Epoch 2/15
574/574 [==============================] - 7s 13ms/step - loss: 0.0471 - custom_loss: 0.0471 - val_loss: 0.0476 - val_custom_loss: 0.0476
Epoch 3/15
574/574 [==============================] - 7s 13ms/step - loss: 0.0457 - custom_loss: 0.0457 - val_loss: 0.0467 - val_custom_loss: 0.0467
Epoch 4/15
574/574 [==============================] - 7s 13ms/step - loss: 0.0450 - custom_loss: 0.0450 - val_loss: 0.0461 - val_custom_loss: 0.0461
Epoch 5/15
574/574 [==============================] - 7s 12ms/step - loss: 0.0445 - custom_loss: 0.0445 - val_l

2021-12-11 00:39:28.172148: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-11 00:39:28.172189: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-12-11 00:39:41.746138: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-11 00:39:41.746184: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-11 00:39:41.746199: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ebf4e1cc3c64): /proc/driver/nvidia/version does not exist
2021-12-11 00:39:41.746351: I tensorflow/core/platform/cpu_featu

# Run Test Data Predictions

In [39]:
%%bash 
docker exec autorec python predict.py

Reading prediction data... 
Loading trained model... 
Preprocessing prediction data... 
Making predictions... 
Post-processing predictions for writing... 
Saving predictions... 
Done with predictions.


2021-12-11 00:43:53.865772: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-11 00:43:53.865816: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-12-11 00:44:08.346454: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-11 00:44:08.346497: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-11 00:44:08.346509: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ebf4e1cc3c64): /proc/driver/nvidia/version does not exist
2021-12-11 00:44:08.346635: I tensorflow/core/platform/cpu_featu

# Score Test Data Predictions

In [29]:
%%bash 
docker exec autorec python score.py

scores: {'mse': 17.453816578632445, 'rmse': 4.17777651133141, 'mae': 3.2842097552282747, 'nmae': 4.380761472706451, 'smape': 107.29605599119468, 'r2': 0.377373158133142}


2021-12-10 06:35:19.297206: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-10 06:35:19.297252: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Check Outputs In mounted volume
- data: this is where we mounted training and test data
- logs: contains logged model training output
- model: trained model artifacts
- output: contains predictions. Also contains output from hyper-parameter tuning, if run. 

In [30]:
%%bash
ls -R ml_vol

ml_vol:
data
logs
model
output

ml_vol/data:
test
train

ml_vol/data/test:
ratings_test.csv

ml_vol/data/train:
attribute_defn.csv
ratings_train.csv
user_attributes.csv

ml_vol/logs:

ml_vol/model:
checkpoint
model_params
model_weights.data-00000-of-00001
model_weights.index
preprocess_pipe.save

ml_vol/output:
predictions.csv
scores.csv


# Push to docker hub 
Need permission to push to this repository on docker hub

In [32]:
%%bash
docker push abudesai/rec_base_autorec:v1.0.0

The push refers to repository [docker.io/abudesai/rec_base_autorec]
5f70bf18a086: Preparing
19e722538b6d: Preparing
7444b983b4bd: Preparing
6463cb47ff88: Preparing
d82a4c4e92ff: Preparing
155411760e3a: Preparing
2cbb114c7605: Preparing
459d9d53a256: Preparing
831c5620387f: Preparing
155411760e3a: Waiting
2cbb114c7605: Waiting
459d9d53a256: Waiting
831c5620387f: Waiting
d82a4c4e92ff: Mounted from abudesai/rec_base_mf
6463cb47ff88: Mounted from abudesai/rec_base_mf
5f70bf18a086: Mounted from abudesai/rec_base_mf
155411760e3a: Mounted from abudesai/rec_base_mf
459d9d53a256: Mounted from abudesai/rec_base_mf
2cbb114c7605: Mounted from abudesai/rec_base_mf
831c5620387f: Mounted from abudesai/rec_base_mf
19e722538b6d: Pushed
7444b983b4bd: Pushed
v1.0.0: digest: sha256:f3080bee5569a4be2c10521bde06c194397d4ca9f852c29089ee5de8779bb1de size: 2209


# Stop Container and Remove Image

In [ ]:
%%bash
docker stop autorec

In [ ]:
%%bash
docker rm autorec

In [ ]:
%%bash
docker rmi abudesai/rec_base_autorec:v1.0.0